In [1]:
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
import numpy as np
import pandas as pd
import os

In [2]:
# Configura la longitud de la secuencia y el horizonte de predicción
sequence_length = 24  # Por ejemplo, 24 pasos (2h 24min)
horizon = 1           # Predicción 1 paso adelante

# Función para crear ventanas de tiempo
def create_dataset(X, y, sequence_length=24, horizon=1):
    Xs, ys = [], []
    for i in range(len(X) - sequence_length - horizon + 1):
        Xs.append(X[i:(i + sequence_length)])
        ys.append(y[i + sequence_length + horizon - 1])
    return np.array(Xs), np.array(ys)

In [13]:
data_dir = os.path.join('..', 'processed')

X_train_path = os.path.join(data_dir, 'X_train.csv')
X_test_path = os.path.join(data_dir, 'X_test.csv')
y_train_path = os.path.join(data_dir, 'y_train.csv')
y_test_path = os.path.join(data_dir, 'y_test.csv')

# Cargar los datos con pandas
X_train = pd.read_csv(X_train_path)
X_test = pd.read_csv(X_test_path)
y_train = pd.read_csv(y_train_path)
y_test = pd.read_csv(y_test_path)

FileNotFoundError: [Errno 2] No such file or directory: '..\\processed\\X_train.csv'